# MALDI Extraction

This notebook is the analysis pipeline for the MALDI data. For each MALDI run, this notebook runs the following workflow:

- **Spectra extraction**: read m/z spectra and corresponding intensities from the binary files
- **Peak filtering**: identify most prominent m/z peaks, along with their widths, heights, areas, etc.
- **Coordinate integration**: map peak information onto the MALDI slide
- **Glycan matching**: map filtered peaks to master glycan list
- **Core-level analysis (TMAs only)**: crop out specific cores, extract core-level stats

## 0. Import Modules

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyimzml.ImzMLParser import ImzMLParser
from maldi_tools import extraction, plotting

## 1. Define Global Constants

### 1.1. File paths

The following variables are defined pointing to paths in your MALDI run. **Only `base_dir` should be changed, the other folders should maintain the existing names and subdirectory structure.**

* `base_dir`: the path to your MALDI data.
* `imzml_dir`: the path inside `base_dir` to your `imzml` folder. This will contain the `.imzml` and `.ibd` files extracted from SCiLS.
* `library_dir`: the path inside `base_dir` to your `libraries` folder. This will contain the master list to use for glycan matching.
* `extraction_dir`: the path inside `base_dir` to your `extracted` folder. Contains the integrated images for each filtered peak and glycan-matched peak across the slide.
* `debug_dir`: the path inside `base_dir` to your `debug` folder. Individual peak height and width info is saved here for debugging purposes.

In [ ]:
base_dir = pathlib.Path("../data/panc2055")
imzml_dir = base_dir / "imzml"
library_dir = base_dir / "libraries"
extraction_dir = base_dir / "output" / "extracted"
debug_dir = base_dir / "output" / "debug"

Create the directory structure (if it already exists, nothing is overwritten).

In [ ]:
# Create directories
for directory in [base_dir, imzml_dir, library_dir, extraction_dir, debug_dir]:
    if not os.path.exists(directory):
        directory.mkdir(parents=True, exist_ok=True)

**NOTE: At this point, ensure you have the following files in the following locations:**

* **`.imzml`/`.ibd` files**: extracted using SCiLS. **Either explicitly point to the `imzml` subfolder when extracting, or manually copy these files to the `imzml` subfolder afterwards.**
* **Master glycan list**: defining a singular master glycan list is a WIP. For now, ask a lab member for the peak list to use. **This file must be manually copied into the `libraries` subfolder.**

And define the following variable:

* `imzml_file`: the name of the `.imzml` file in the `imzml` subfolder.

In [ ]:
imzml_file = "panc2055.imzml"
imzml_path = imzml_file / imzml_path

### 1.2. Plotting parameters

Define the pltoting parameters.

In [ ]:
plt.rcParams["figure.figsize"] = (20, 13)
plt.rcParams["ytick.color"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["axes.edgecolor"] = "w"
plt.rcParams["axes.facecolor"] = "black"
plt.rcParams["savefig.edgecolor"] = "w"
plt.rcParams["savefig.facecolor"] = "black"
plt.rcParams["figure.facecolor"] = "black"
plt.rcParams["figure.constrained_layout.use"] = False

## 2. Data Loading

### 2.1. Spectra

Load in the spectra information defined by the `.imzml`/`.ibd` file. The following variables are extracted:

* `total_mass_df`: tabulates every m/z value found along with their corresponding intensity (summed across all pixels).
* `thresholds`: defines the nth intensity value (defined by `intensity_percentile`, computed across all m/z values) found across each pixel.

In [ ]:
# define the .imzml/.ibd loader object
imz_data = ImzMLParser(imzml_path, include_spectra_metadata="full")

# extract the spectra and threshold array
total_mass_df, thresholds = extraction.extract_spectra(
    imz_data=imz_data, intensity_percentile=intensity_percentile
)

# define the global intensity threshold
intensity_percentile = 99
global_intensity_threshold = np.percentile(total_mass_df["intensity"].values, intensity_percentile)
print(f"Global Intensity Threshold: {global_intensity_threshold}")

# display the format and the top few peaks extracted
display(total_mass_df.head())

For additional verification, set `largest_intensity_count` to see the `n` largest intensities.

In [ ]:
largest_intensity_count = 10
total_mass_df.nlargest(largest_intensity_count, ["intensity"])

### 2.2. Master library peaks

Load the master glycan peak list, this will be used by the library matching process.

In [ ]:
library_peak_list = library_dir / "glycan_peaklist_KL.csv"
library_peak_df = pd.read_csv(library_peak_list)

# visualize the top few master peaks defined
library_peak_df.head()

## 3. Peak Analysis

### 3.1. Define prominence thresholds

Although various peaks can be identified across a run's spectra, the vast majority of them will be too small to indicate any meaningful signal. To address this, a prominence-based peak filtering method is used (see https://www.mathworks.com/help/signal/ug/prominence.html for a good definition of prominence).

The first step is to use a rolling window-based approach to extract the prominence thresholds to use for peak filtering.

In [ ]:
log_intensities, log_int_percentile = extraction.rolling_window(
    total_mass_df=total_mass_df, intensity_percentile=intensity_percentile, window_size=5000
)

Visualizes how the thresholds you chose affect the peak candidates identified.

In [ ]:
plotting.plot_intensities(
    total_mass_df=total_mass_df,
    log_intensities=log_intensities,
    log_int_percentile=log_int_percentile,
    global_intensity_threshold=global_intensity_threshold,
)

### 3.2. Extract and filter the m/z peaks

Once you're happy with the prominence thresholds defined in `log_int_percentile`, run the following cell to identify the m/z peaks, which does the following:

1. A traditional local maxima-based approach applies the first filter
2. For the remaining candidates, the `log_int_percentile` prominence thresholds apply a second filter to remove insignificant peaks

In [ ]:
peak_candidate_idxs, peak_candidates = extraction.signal_extraction(
    total_mass_df=total_mass_df, log_int_percentile=log_int_percentile
)

print(f"Candiate Peak Count: {len(peak_candidates)}")

Visualize the discovered peaks.

In [ ]:
plotting.plot_discovered_peaks(
    total_mass_df=total_mass_df,
    peak_candidate_idxs=peak_candidate_idxs,
    peak_candidates=peak_candidates,
    global_intensity_threshold=global_intensity_threshold,
)

### 3.3. Compute peak widths

For each peak, compute the corresponding width at 10% of the height defined from the peak's base. This will be necessary for coordinate integration (WIP).

In [ ]:
peak_df, l_ips_r, r_ips_r, peak_widths_height = extraction.get_peak_widths(
    total_mass_df=total_mass_df,
    peak_candidate_idxs=peak_candidate_idxs,
    peak_candidates=peak_candidates,
    thresholds=thresholds,
)

### 3.4. Save Peak Spectra

Define the m/z value of each peak, along with their corresponding lower and upper m/z bounds.

* `save_peak_spectra_debug`: whether to save the corresponding peak spectra graphs to the `debug` folder. We highly recommend leaving this as `True`.

In [ ]:
save_peak_spectra_debug = True

panel_df = extraction.peak_spectra(
    total_mass_df=total_mass_df,
    peak_df=peak_df,
    peak_candidate_idxs=peak_candidate_idxs,
    peak_candidates=peak_candidates,
    peak_widths_height=peak_widths_height,
    l_ips_r=l_ips_r,
    r_ips_r=r_ips_r,
    save_peak_spectra_debug=save_peak_spectra_debug,
    debug_dir=debug_dir,
)

display(panel_df.head())

## 4. Coordinate Integration

Once peaks have been identified, we need a way of mapping this information back to the slide. Across a coordinate's m/z spectrum, if it is also an identified peak from step 3, we store the corresponding intensity at that coordinate.

In [ ]:
image_data = extraction.coordinate_integration(peak_df=peak_df, imz_data=imz_data)

For QC purposes, visualize the intensity distribution around a desired peak

* `desired_peak_hist`: the peak around where you want to visualize corresponding intensities (ideally something from your library)
* `bin_count`: number of bins to use for the histogram

In [ ]:
desired_peak_hist = 1809.639659
bin_count = 40

_ = image_data.sel(peak=[desired_peak_hist], method="nearest").plot.hist(bins=bin_count)

Save the integrated intensity images per peak to the `extracted` folder. For every peak, a float32 and int32 image will be saved to the `float` and `int` subdirectories respectively.

* The `float` images should be used for quantitative downstresam analysis
* The `int` images are saved for visualization, as they're more compatible with most image viewers

In [ ]:
plotting.save_peak_images(image_xr=image_data, extraction_dir=extraction_dir)

## 5. Glycan Library Matching

While the filtered peaks provide meaningful information, they're not particularly useful without knowing what glycan encompasses them. The master glycan library list (`library_peak_df`) defines the glycans of interest as well as the m/z value they're centered at. In this way, peak values can be mapped to their corresponding glycan within a tolerance range.

In [ ]:
matched_peaks_df = extraction.library_matching(
    image_xr=image_data, library_peak_df=library_peak_df, ppm=50, extraction_dir=extraction_dir
)

As with the original peak images, the library matched intensity images are also saved, this time to the `output/extracted/library_matched` folder. There will likewise be `float` and `int` subdirectories containing float32 and int32 representations.

In [ ]:
plotting.save_matched_peak_images(
    image_xr=image_data, matched_peaks_df=matched_peaks_df, extraction_dir=extraction_dir
)

## 6. Core Naming and Cropping

For TMAs, each core is extracted all at once. However, this makes it difficult to locate the exact positions of each core. Additionally, the default names assigned to each core aren't particularly useful because they don't contain any information about their position on the TMA.

This section will help you assign informative names to each core and afterwards, segment out the locations of specific cores to generate FOV-level statistics.

It is helpful first to create an all-encompassing mask that defines the locations of all the cores. This will make it clear where the TMA was scanned for the naming step. You will need to provide the path to one of your extracted glycan images first.

* `glycan_img_path`: path to one glycan image, needed to properly dimension the mask
* `glycan_mask_path`: where the mask will be saved

In [ ]:
glycan_img_path = "path/to/glycan_img.tiff"
glycan_mask_path = "path/to/glycan_mask.png"

# generate and save the glycan mask
extraction.generate_glycan_mask(
    imz_data=imz_data,
    glycan_img_path=glycan_img_path,
    glycan_mask_path=glycan_mask_path
)

Each core on the TMA should be appropriately named by the <a href=https://tsai.stanford.edu/research/maldi_tma/>TSAI MALDI tiler</a>. You will need to provide the PNG saved at `glycan_mask_path` as input. **Ensure that this step is completed before running the following sections.**

The poslog files for your TMA run will contain each scanned coordinate in the exact order it was scanned. This, along with the tiler output, will be needed to map each coordinate to its respective core.

* `centroid_path`: TSAI MALDI tiler output, contains name of each core mapped to respective centroid
* `poslog_paths`: list of poslog files used for the TMA, contains all coordinates in order of acquisition. **Make sure this matches up with the order of acquisition for your run.**

In [ ]:
centroid_path = "path/to/centroids.json"
poslog_paths = ["path/to/poslog1.txt", "path/to/poslog2.txt"]

# map coordinates to core names
region_core_info = extraction.map_coordinates_to_core_name(
    imz_data=imz_data,
    centroid_path=centroid_path,
    poslog_paths=poslog_paths
)

To generate FOV-level statistics, an individual mask for each core named by TSAI will be saved. They can then be loaded in as needed in the FOV-level-statistic-generating functions.

* `glycan_crop_save_dir`: the directory where these masks will be saved

In [ ]:
glycan_crop_save_dir = "path/to/glycan/crops"
if not os.path.exists(glycan_crop_save_dir):
    os.makedirs(glycan_crop_save_dir)

extraction.generate_glycan_crop_masks(
    glycan_mask_path=glycan_mask_path,
    region_core_info=region_core_info,
    glycan_crop_save_dir=glycan_crop_save_dir
)

Run the following cell to visualize the masks for certain cores for testing.

* `cores_to_crop`: define all the cores you want to visualize their masks for. If multiple cores are specified, the individual masks are combined. Set to `None` to crop all cores out.

In [ ]:
cores_to_crop = ["R1C1", "R1C2"]

# extract a binary mask with just the cores specified
core_cropping_mask = extraction.load_glycan_crop_masks(
    glycan_crop_save_dir=glycan_crop_save_dir,
    cores_to_crop=cores_to_crop
)

# visualize the mask
_ = plt.imshow(core_cropping_mask)